# Microphysics Manifold 3D Visualization

## Detailed Analysis

This script generates a three-dimensional visualization of the internal physical properties of the generated neutron stars. Unlike the macroscopic plots (which show Mass, Radius, and Tidal Deformability), this script focuses on the **microscopic** features used by the advanced machine learning models (specifically Model D).

The visualization plots the stars in a phase space defined by Central Density, Central Speed of Sound, and the Topological Slope of the Mass-Radius curve. To make the complex 3D structure interpretable, 2D density contours (Kernel Density Estimates) are projected onto the "walls" of the plot axes. This allows the correlations between pairs of variables to be viewed simultaneously with the full 3D distribution.

## Physics and Math

The phase space $\mathcal{P}$ visualized in this script consists of three microphysical parameters:

1.  **Central Energy Density ($\varepsilon_c$):** The density at the core of the star.
2.  **Central Speed of Sound ($c_s^2$):** The stiffness of the Equation of State at the core, defined as $c_s^2 = dP/d\epsilon$.
3.  **Topological Slope ($dR/dM|_{1.4}$):** The derivative of the radius with respect to mass at a canonical mass of $1.4 M_{\odot}$.

### Density Projections
To visualize the structure of the data on the 2D axis planes, the probability density function $P(x,y)$ is estimated using Gaussian Kernel Density Estimation (KDE):

$$
\hat{f}(x, y) = \frac{1}{n h^2} \sum_{i=1}^{n} K\left(\frac{x-x_i}{h}, \frac{y-y_i}{h}\right)
$$

where $K$ is the Gaussian kernel and $h$ is the bandwidth. These densities are then plotted as contours on the bounding box walls (e.g., the $\varepsilon_c - c_s^2$ plane), effectively showing the "shadow" of the 3D data.

## Code Walkthrough

### 1. Data Preparation and Grid Generation
The script first filters the dataset to ensure all three required columns (`Eps_Central`, `CS2_Central`, `Slope14`) are present.

A helper function, `get_density_grid`, is defined to compute the KDE for the projections. It creates a meshgrid over the specified axis limits and evaluates the Gaussian KDE on that grid.

```python
def get_density_grid(x, y, x_lim, y_lim, grid_size=60):
    # ...
    kernel = gaussian_kde(np.vstack([x_s, y_s]))
    Z = np.reshape(kernel(positions).T, XX.shape)
    return XX, YY, Z / Z.max()
```

### 2. Static 3D Plotting (Matplotlib)
The static PDF figure is generated using `mpl_toolkits.mplot3d`.
*   **Scatter Cloud:** A random sample of points is plotted as small, semi-transparent markers to visualize the volume occupied by Hadronic (Green) and Quark (Magenta) models.
*   **Wall Projections:** The `plot_mpl_wall` function is used to project the 2D densities onto the plot boundaries.

```python
# Floor (z=Min Slope): Eps vs CS2
plot_mpl_wall(ax, h_data['Eps_Central'], h_data['CS2_Central'], 
              q_data['Eps_Central'], q_data['CS2_Central'], 
              'z', slope_lim[0], eps_lim, cs2_lim)
```

By setting the `zdir` (direction) and `offset` arguments in `contourf`, the contours are drawn flat against the specified axis limit (the "floor" or "walls").

### 3. Interactive 3D Plotting (Plotly)
If the `plotly` library is available, an interactive HTML file is generated. This allows the user to rotate and zoom into the manifold.
*   **Scatter3d:** Used for the data points.
*   **Surface:** Used for the wall projections. Since Plotly does not have a direct "wall contour" feature like Matplotlib, surfaces are created on the boundary planes, colored by the KDE density values computed earlier.

## Visualization Output

The script produces `plots/fig_microphysics_3d.pdf` (and optionally an HTML file).

*   **X-Axis:** Central Energy Density $\varepsilon_c$ [MeV/fm$^3$].
*   **Y-Axis:** Central Speed of Sound Squared $c_s^2$.
*   **Z-Axis:** Slope $dR/dM$ at $1.4 M_{\odot}$.
*   **Visual Elements:**
    *   **Scatter Cloud:** Hadronic stars (Green) cluster at lower densities and lower sound speeds, with negative slopes. Quark stars (Magenta) extend to higher densities and sound speeds, often with positive slopes.
    *   **Projections (Shadows):**
        *   **Floor ($\varepsilon_c$ vs $c_s^2$):** Shows the correlation between density and stiffness.
        *   **Back Wall ($\varepsilon_c$ vs Slope):** Shows how the slope transition occurs as density increases.
        *   **Side Wall ($c_s^2$ vs Slope):** Highlights that positive slopes are associated with high sound speeds ($c_s^2 > 1/3$).